In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
# Генерируем колонки по шаблону
def generate_columns(n, name):
    columns = [name+str(i) for i in range(1, n+1)]
    return columns

In [3]:
# Убираем лишние нолики, чтобы красивее было смотреть
def drop_zero(n, except_k):
    cols = set(generate_columns(n, "#host")) | set(generate_columns(n, "POShost"))
    drop = cols - {f"#host{except_k}", f"POShost{except_k}"}
    return list(drop)

In [4]:
# Смотрим на конструкции по нужным столбцам и слову
# Можно попросить показать прочерки (outhost=True)
# Можно смотреть только на одну конкретную связь (host_of=номер слова, на связь которого смотрим)
# Можно задать верхнюю и нижнюю границы отношения
# Можно дать дф, из которого надо смотреть, можно файлы all_with_ratios
def watch_constr(n, cols="all", host_of=None, bott_lim=0.0, up_lim=1.0, outhost=False, entries_lim=0, source="file"):
    if isinstance(source, pd.DataFrame):
        df = source
    else:
        df = pd.read_csv(source, sep=",", low_memory=False)
    
    search_patt = ((df["ratio"] >= bott_lim) & 
                  (df["ratio"] <= up_lim) &
                  (df["total_entries"] >= entries_lim))
    if cols == "all":
        cols = ["any"] * n
    for i in range(n):
        if cols[i] != "any":
            search_patt &= (df[f"POS{i+1}"] == cols[i])
        if not outhost:
            search_patt &= (df[f"POShost{i+1}"] != '_')
    if host_of:
        search_patt &= (df[f"#host{host_of}"] != "0")
        return df[search_patt].drop(columns=drop_zero(n, host_of))
    else:
        return df[search_patt]

In [6]:
# Удаляет из df его почти*_кусочек subdf
# *почти -- потому что колонки не совпадают, для этого мерджим
# Возвращает обновленный new_df
def delete_subdf(df, subdf):
    del_df = pd.merge(subdf, df, on=list(subdf.columns))
    new_df = pd.concat([df, del_df]).drop_duplicates(keep=False)
    return new_df

In [7]:
# Передвигает названия колонок и номера связей
# n -- размер n-граммы
# df -- датафрейм, в котором нужно передвинуть колонки
# shift_by -- на сколько значений передвинуть, по умолчанию на 1
def shift_to_r(n, df, shift_by=1):
    df_copy = df.copy(deep=True)
    df_copy.columns = [col_name[:-1] + str(int(col_name[-1]) + 1) for col_name in list(df_copy.columns)]
    for x in range (shift_by + 1, n + shift_by):
        df_copy[f"#host{x}"] = df_copy[f"#host{x}"].apply(lambda x: str(int(x) + 1) if x != "0" else "0")
        
    return df_copy

In [8]:
# Избавляемся от лишних на данном этапе анализа колонок
def drop_add_info(df, n):
    return df.drop(["#construction", "ratio"] + generate_columns(n, "POShost"), axis=1)

In [10]:
# Расширение уже обработанных n-грамм до n+1-грамм
def already(cool_now, df_new, n):
    # Уже есть + доп контекст справа
    already_r = pd.merge(cool_now, df_new, on=list(cool_now.columns))
    # Уже есть + доп контекст слева
    cool_now_l = shift_to_r(n, cool_now)
    already_l = pd.merge(cool_now_l, df_new, on=list(cool_now_l.columns))
    
    already = pd.concat([already_r, already_l], ignore_index=True)
    already = drop_add_info(already, n).drop(["total_entries"], axis=1)
    return already

In [11]:
# Правильный порядок для записи
def right_col_order(n):
    right_order = []
    for pos, host in zip(generate_columns(n, "POS"), generate_columns(n, "#host")):
        right_order.extend([pos, host])
    right_order.append("total_entries")
    return right_order

In [19]:
# Спасение конструкций за счёт расширения контекста
def save(n, search_in, cool_now, meh_now, side):

    # Добавленная колонка и расширяемый датафрейм в зависимости от направления расширения
    if side == "left":
        meh_now = shift_to_r(n, meh_now)
    
    # Расширение контекста и отбор по порогу
    saved = pd.merge(search_in, meh_now, on=list(meh_now.columns))
    saved = saved[saved["ratio"] >= 0.97]
    
    # Обновляем область поиска
    search_in = delete_subdf(search_in, saved)
    saved = drop_add_info(saved, n)
    
    # Записываем в файл
    add_cool(n, saved, source=path + f"all_cool_{n}.csv")
    
    # Обновляем список уже обработанных
    cool_now = pd.concat([cool_now, saved.drop(["total_entries"], axis=1)])
    
    # Обновляем список тех, кого надо спасать
    meh_now = pd.concat([meh_now, saved])[list(meh_now.columns)].drop_duplicates(keep=False)
    
    return search_in, cool_now, meh_now

In [13]:
# Создаем csv файл для дальнейшей записи по кусочкам
def init_csv(n, file_dir):
    with open(file_dir + f"all_cool_{n}.csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',')
        csvwriter.writerow(right_col_order(n))

In [14]:
# Добаляем в csv файл полученные на каком-то этапе конструкции
def add_cool(n, cool, source):
    with open(source, 'a') as csvadd:
        cool.to_csv(csvadd, header=False, index=None, line_terminator='\n')

In [15]:
def extract_cool(n, path):
    init_csv(n, path)
    if n == 3:
        cool_now = watch_constr(n, "all", bott_lim=0.97, source=path + f"all_with_ratios_{n}.csv")
        cool_now = drop_add_info(cool_now, n)
        
        meh_now = watch_constr(n, "all", up_lim=0.9699, source=path + f"all_with_ratios_{n}.csv")
        meh_now = drop_add_info(meh_now, n)
        
        add_cool(n, cool_now, source=path + f"all_cool_{n}.csv")
        # Не удаляем total_entries сразу же, тк нужен для записи конструкций
        return cool_now.drop(["total_entries"], axis=1), meh_now.drop(["total_entries"], axis=1)
        
    else:
        cool_now, meh_now = extract_cool(n-1)
        
        # Из n датафрейма удаляем уже разобранные n-1 конструкции
        df_new = watch_constr(n, "all", source=path + f"all_with_ratios_{n}.csv")
        cool_now = already(cool_now, df_new, n)
        
        search_in = delete_subdf(df_new, cool_now)
        print(search_in.columns)
        
        # Добавляем и сохраняем справа и слева
        for side in ["right", "left"]:
            search_in, cool_now, meh_now = save(n, search_in, cool_now, meh_now, side)
        
        # Оставшиеся после расширений хорошие сами по себе n-граммы
        pure_cool = watch_constr(n, "all", bott_lim=0.97, source=search_in)
        pure_cool = drop_add_info(pure_cool, n)
        add_cool(n, pure_cool, source=path + f"all_cool_{n}.csv")
        
        cool_now = pd.concat([cool_now, pure_cool.drop(["total_entries"], axis=1)])
        meh_now = watch_constr(n, "all", up_lim=0.9699, source=search_in)
        meh_now = drop_add_info(meh_now, n).drop(["total_entries"], axis=1)
        
        return cool_now, meh_now

In [20]:
# Требуется большая оперативная память, но в колабе работает
paths = ["./outcome files/dev_train_test/",
         "./outcome files/dev_train/"]
# for path in paths:
    # extract_cool(6, path)

In [17]:
# объединение
def sum_up(n, source):
    cols = generate_columns(n, "POS")
    df = pd.read_csv(source+f"all_cool_{n}.csv", sep=",", low_memory=False)
    df_constr = df.groupby(cols)[generate_columns(n, "#host")].sum()
    df_constr.reset_index(inplace=True)
    df_min_entr = df.groupby(cols)["total_entries"].min().reset_index(name="total_entries")
    df_final = pd.merge(df_constr, df_min_entr, on=cols)
    df_final = df_final[right_col_order(n)]
    df_final.to_csv(source + f"all_cool_{n}_grouped.csv", index=None, line_terminator='\n')

In [252]:
for i in range(3, 7):
    sum_up(i, "./outcome files/dev_train_test/")
    sum_up(i, "./outcome files/dev_train/")